# Popcorn Fundraiser Sales Allocation

This notebook splits shared popcorn sales equally among scouts present per shift.

Inputs:
- shifts.csv
- sales.csv

Rules:
- Uses Date Taken as the authoritative sales date
- Matches on Date + Shift
- Splits sales equally among scouts present


In [36]:
# =============================================================================
# CONFIGURATION - Edit file paths using the widgets below
# =============================================================================
import ipywidgets as widgets
from IPython.display import display

# Create text input widgets with defaults
shifts_input = widgets.Text(
    value='data-example/shifts.csv',
    description='Shifts file:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='500px')
)

sales_input = widgets.Text(
    value='data-example/sales.csv',
    description='Sales file:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='500px')
)

display(shifts_input, sales_input)

Text(value='data-example/shifts.csv', description='Shifts file:', layout=Layout(width='500px'), style=TextStyl…

Text(value='data-example/sales.csv', description='Sales file:', layout=Layout(width='500px'), style=TextStyle(…

In [37]:
import pandas as pd

In [38]:
# --- Clean shifts ---
shifts = pd.read_csv(shifts_input.value)
shifts_clean = shifts.copy()
shifts_clean['Date'] = pd.to_datetime(shifts_clean['Date']).dt.date
shifts_clean['Shift'] = shifts_clean['Shift'].str.replace(' US/Central','', regex=False)

shifts_clean.head()

,Date,Site Name,Address Line 1,Shift,Scout Name,Scout Code,Scout Email,Scout Phone
0,2025-09-13,Bass Pro Shops,20000 Bass Pro Dr,10:00 AM - 11:00 AM,Alex M,ABC12345,alex.m@example.com,2015551234
1,2025-09-13,Bass Pro Shops,20000 Bass Pro Dr,10:00 AM - 11:00 AM,Jordan T,XYZ67890,jordan.t@example.com,2015559999
2,2025-09-13,Bass Pro Shops,20000 Bass Pro Dr,11:00 AM - 12:00 PM,Alex M,ABC12345,alex.m@example.com,2015551234
3,2025-09-13,Bass Pro Shops,20000 Bass Pro Dr,11:00 AM - 12:00 PM,Casey L,DEF34567,casey.l@example.com,2015558888
4,2025-09-13,Lowe's,29645 Frederick Blvd,02:00 PM - 03:00 PM,Jordan T,XYZ67890,jordan.t@example.com,2015559999


In [39]:
# --- Clean sales (use Date Taken, Storefront only) ---
sales = pd.read_csv(sales_input.value)
sales_clean = sales.copy()
sales_clean = sales_clean[sales_clean['Sale Type'] == 'Storefront']
sales_clean['Date'] = pd.to_datetime(sales_clean['Date Taken']).dt.date
sales_clean['Shift'] = sales_clean['Shift'].str.replace(' US/Central','', regex=False)
sales_clean['Shift'] = sales_clean['Shift'].str.replace(r':\d{2}(?=\s)', '', regex=True)

sales_clean['Total Order Amount'] = (
	sales_clean['Total Order Amount']
	.astype(str)
	.str.replace(r'[$,]', '', regex=True)
	.astype(float)
)

sales_clean.head()

,Order Number,Scout,Scout Code,Total Order Amount,Payment Type,Sale Type,Date Taken,Status Code,Site Name:Date,Site Address Line 1,Site City,Shift,Customer Name,Customer Address Line 1,Customer Address Line 2,Customer City,Customer Zip,Customer Phone,Customer Email,Date
0,13898808-app,Alex M,ABC12345,40.0,CREDIT,Storefront,2025-09-13,Delivered,Bass Pro Shops: 2025-09-13,20000 Bass Pro Dr,Spanish Fort,10:00 AM - 11:00 AM,John Smith,123 Main St,NaN,Montgomery,36104,2015551111,john.smith@example.com,2025-09-13
1,13901591-app,Alex M,ABC12345,20.0,CREDIT,Storefront,2025-09-13,Delivered,Bass Pro Shops: 2025-09-13,20000 Bass Pro Dr,Spanish Fort,10:00 AM - 11:00 AM,Jane Doe,456 Oak Ave,NaN,Birmingham,35203,2015552222,jane.doe@example.com,2025-09-13
2,13902692-app,Jordan T,XYZ67890,50.0,CREDIT,Storefront,2025-09-13,Delivered,Bass Pro Shops: 2025-09-13,20000 Bass Pro Dr,Spanish Fort,10:00 AM - 11:00 AM,Mike Johnson,789 Pine Rd,NaN,Tuscaloosa,35401,2015553333,mike.johnson@example.com,2025-09-13
3,13903484-app,Alex M,ABC12345,30.0,CREDIT,Storefront,2025-09-13,Delivered,Bass Pro Shops: 2025-09-13,20000 Bass Pro Dr,Spanish Fort,11:00 AM - 12:00 PM,Sarah Lee,321 Elm St,NaN,Huntsville,35801,2015554444,sarah.lee@example.com,2025-09-13
4,13905678-app,Casey L,DEF34567,25.0,CREDIT,Storefront,2025-09-13,Delivered,Bass Pro Shops: 2025-09-13,20000 Bass Pro Dr,Spanish Fort,11:00 AM - 12:00 PM,Tom Brown,654 Maple Dr,NaN,Dothan,36301,2015555555,tom.brown@example.com,2025-09-13


In [40]:
# Aggregate sales per shift
sales_by_shift = (
	sales_clean
	.groupby(['Date','Shift'], as_index=False)
	['Total Order Amount']
	.sum()
	.rename(columns={'Total Order Amount':'Shift Total Sales'})
)

sales_by_shift.head()

,Date,Shift,Shift Total Sales
0,2025-09-13,02:00 PM - 03:00 PM,45.0
1,2025-09-13,10:00 AM - 11:00 AM,110.0
2,2025-09-13,11:00 AM - 12:00 PM,55.0


In [41]:
# Count scouts per shift
scouts_per_shift = (
	shifts_clean
	.groupby(['Date','Shift'], as_index=False)
	['Scout Name']
	.nunique()
	.rename(columns={'Scout Name':'Scouts Present'})
)

scouts_per_shift

,Date,Shift,Scouts Present
0,2025-09-13,02:00 PM - 03:00 PM,1
1,2025-09-13,10:00 AM - 11:00 AM,2
2,2025-09-13,11:00 AM - 12:00 PM,2


In [42]:
# Merge and calculate per-scout share
shift_totals = sales_by_shift.merge(
	scouts_per_shift,
	on=['Date','Shift'],
	how='left'
)

per_scout = shifts_clean.merge(
	shift_totals,
	on=['Date','Shift'],
	how='left'
)

per_scout['Scout Share'] = (per_scout['Shift Total Sales'] / per_scout['Scouts Present']).round(2)

per_scout.head()

,Date,Site Name,Address Line 1,Shift,Scout Name,Scout Code,Scout Email,Scout Phone,Shift Total Sales,Scouts Present,Scout Share
0,2025-09-13,Bass Pro Shops,20000 Bass Pro Dr,10:00 AM - 11:00 AM,Alex M,ABC12345,alex.m@example.com,2015551234,110.0,2,55.0
1,2025-09-13,Bass Pro Shops,20000 Bass Pro Dr,10:00 AM - 11:00 AM,Jordan T,XYZ67890,jordan.t@example.com,2015559999,110.0,2,55.0
2,2025-09-13,Bass Pro Shops,20000 Bass Pro Dr,11:00 AM - 12:00 PM,Alex M,ABC12345,alex.m@example.com,2015551234,55.0,2,27.5
3,2025-09-13,Bass Pro Shops,20000 Bass Pro Dr,11:00 AM - 12:00 PM,Casey L,DEF34567,casey.l@example.com,2015558888,55.0,2,27.5
4,2025-09-13,Lowe's,29645 Frederick Blvd,02:00 PM - 03:00 PM,Jordan T,XYZ67890,jordan.t@example.com,2015559999,45.0,1,45.0


Grand total per scout based on total sales allocated to them.

In [43]:
# Grand totals per scout
scout_totals = (
	per_scout
	.groupby('Scout Name', as_index=False)
	['Scout Share']
	.sum()
	.sort_values('Scout Share', ascending=False)
)

scout_totals

,Scout Name,Scout Share
2,Jordan T,100.0
0,Alex M,82.5
1,Casey L,27.5


Now we calculate the amount owed to each scout based on the percentage of sales the Pack receives. The percentage is 35%.

In [44]:
# Calculate 35% share
scout_totals_pct = scout_totals.copy()
scout_totals_pct['Scout Share'] = (scout_totals_pct['Scout Share'] * 0.35).round(2)

print(scout_totals_pct.to_csv(index=False))

Scout Name,Scout Share
Jordan T,35.0
Alex M,28.87
Casey L,9.62



## Notes
- If Scouts Present is NaN, it means the shift date does not exist in shifts.csv
- You can switch to day-level pooling by grouping only on Date
- Export results using DataFrame.to_csv() as needed
